# ISIC2019



Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import sys
import time
import gc
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
import copy
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 7]

from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

from efficientnet_pytorch import EfficientNet
import torchextractor as tx

from itertools import chain, combinations

def get_combs(l):
    return list(chain.from_iterable(combinations(l, r) for r in range(1, len(l)+1)))

In [2]:
sys.path.append('..')

from utils.train import train
from utils.metrics import get_scores, get_metrics
from utils.dataset import get_data_loader
from utils.models import get_model, BaseMetaModel, MetaModel

# Dataset

In [3]:
df = pd.read_csv('train_metadata.csv')
df

,image,diagnostic,age_approx,female,male,anterior_torso,head_neck,lateral_torso,lower_extremity,oral_genital,palms_soles,posterior_torso,upper_extremity,diagnostic_number,folder
0,ISIC_0000000,NV,55.0,1,0,1,0,0,0,0,0,0,0,5,0
1,ISIC_0000001,NV,30.0,1,0,1,0,0,0,0,0,0,0,5,1
2,ISIC_0000002,MEL,60.0,1,0,0,0,0,0,0,0,0,1,4,2
3,ISIC_0000003,NV,30.0,0,1,0,0,0,0,0,0,0,1,5,3
4,ISIC_0000004,MEL,80.0,0,1,0,0,0,0,0,0,1,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25326,ISIC_0073247,BCC,85.0,1,0,0,1,0,0,0,0,0,0,1,26
25327,ISIC_0073248,BKL,65.0,0,1,1,0,0,0,0,0,0,0,2,27
25328,ISIC_0073249,MEL,70.0,0,1,0,0,0,1,0,0,0,0,4,28
25329,ISIC_0073251,NV,55.0,1,0,0,0,0,0,0,1,0,0,5,29


In [4]:
df.columns

Index(['image', 'diagnostic', 'age_approx', 'female', 'male', 'anterior_torso',
       'head_neck', 'lateral_torso', 'lower_extremity', 'oral_genital',
       'palms_soles', 'posterior_torso', 'upper_extremity',
       'diagnostic_number', 'folder'],
      dtype='object')

In [5]:
open_file = open('train_idcs', "rb")
train_folds = pickle.load(open_file)
open_file.close()

open_file = open('val_idcs', "rb")
val_folds = pickle.load(open_file)
open_file.close()

open_file = open('test_idcs', "rb")
test_idcs = pickle.load(open_file)
open_file.close()

## Metadata columns

In [6]:
# metacols_dict = {meta_type: [col  for prefix in meta_types_dict[meta_type] for col in df.columns if col.startswith(prefix)] for meta_type in  ['history', 'habits', 'lesion']}
# metacols_dict

In [7]:
# metacols_dict = {prefix: [col for col in df.columns if col.startswith(prefix)] for prefix in meta_prefixes}
# #metacols_dict

In [8]:
usemeta = ['age', 'gender', 'region']
metacols_dict = {'age': ['age_approx'], 'gender': ['female', 'male'], 'region': ['anterior_torso',
       'head_neck', 'lateral_torso', 'lower_extremity', 'oral_genital',
       'palms_soles', 'posterior_torso', 'upper_extremity']}

# Training

In [9]:
metacombs = get_combs(usemeta)
len(metacombs)

7

In [10]:
combcols_dict = {comb: [col for meta in comb for col in metacols_dict[meta]] for comb in metacombs}
combcols_dict

{('age',): ['age_approx'],
 ('gender',): ['female', 'male'],
 ('region',): ['anterior_torso',
  'head_neck',
  'lateral_torso',
  'lower_extremity',
  'oral_genital',
  'palms_soles',
  'posterior_torso',
  'upper_extremity'],
 ('age', 'gender'): ['age_approx', 'female', 'male'],
 ('age', 'region'): ['age_approx',
  'anterior_torso',
  'head_neck',
  'lateral_torso',
  'lower_extremity',
  'oral_genital',
  'palms_soles',
  'posterior_torso',
  'upper_extremity'],
 ('gender', 'region'): ['female',
  'male',
  'anterior_torso',
  'head_neck',
  'lateral_torso',
  'lower_extremity',
  'oral_genital',
  'palms_soles',
  'posterior_torso',
  'upper_extremity'],
 ('age', 'gender', 'region'): ['age_approx',
  'female',
  'male',
  'anterior_torso',
  'head_neck',
  'lateral_torso',
  'lower_extremity',
  'oral_genital',
  'palms_soles',
  'posterior_torso',
  'upper_extremity']}

In [11]:
torch.cuda.device_count()

4

In [12]:
#model_names = ['resnet18']
model_names = ['resnet50', 'effnetb3', 'resnext50', 'vgg11', 'vit_b_32']


In [13]:
# fusion_methods = ['concat', 'metanet', 'metablock']
fusion_methods = ['concat']

In [14]:
data_dir      = 'imgs/ISIC_2019_Training_Input'
# batch_size    = 16
batch_size    = 32

num_workers   = 16
input_size    = 224

train_transform = transforms.Compose([transforms.RandomResizedCrop(input_size),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

val_transform   = transforms.Compose([transforms.Resize((input_size, input_size)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [15]:
#n_classes = len(set(train_labels))
n_epochs  = 100


lr        = 1e-3 # Learning rate
device    = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

n_samples = df.diagnostic_number.value_counts().sort_index().values

weights = [1 - (x / sum(n_samples)) for x in n_samples]
weights = torch.FloatTensor(weights).to(device)

criterion = nn.CrossEntropyLoss(weight=weights).to(device)

saved_models_folder      = 'saved_comb_models'
saved_scores_folder      = 'saved_comb_scores'
saved_base_models_folder = 'saved_basemodels'
saved_base_scores_folder = 'saved_basescores'

cuda


## Training modelos concat

In [ ]:
#folds = [0, 1, 2, 3, 4]
#folds = [1, 2, 3, 4]
folds  = [0]
n_reducer_block = 256

for fold in folds:
    for metacomb in metacombs:
        metacomb_name = '_'.join(list(metacomb))
        
        metadata_cols = combcols_dict[metacomb]
        
         # Dataloaders
        train_idcs = train_folds[fold]
        val_idcs   = val_folds[fold]
        train_imgs = df.loc[train_idcs, 'image'].values
        val_imgs   = df.loc[val_idcs, 'image'].values
        test_imgs  = df.loc[test_idcs, 'image'].values

        train_paths = [f'{os.path.join(data_dir, img)}.jpg' for img in train_imgs]
        val_paths   = [f'{os.path.join(data_dir, img)}.jpg' for img in val_imgs]
        test_paths  = [f'{os.path.join(data_dir, img)}.jpg' for img in test_imgs]

        train_labels = df.loc[train_idcs, 'diagnostic_number'].values
        val_labels   = df.loc[val_idcs,   'diagnostic_number'].values
        test_labels  = df.loc[test_idcs,  'diagnostic_number'].values

        train_metadata = df.loc[train_idcs, metadata_cols].values
        val_metadata   = df.loc[val_idcs, metadata_cols].values
        test_metadata  = df.loc[test_idcs, metadata_cols].values
        train_dataloader = get_data_loader(train_paths, train_labels, metadata=train_metadata, transform=train_transform, batch_size=batch_size, num_workers=num_workers)
        val_dataloader   = get_data_loader(val_paths, val_labels, metadata=val_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers)
        test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 

        # Training
        n_classes = len(set(train_labels))
        n_metadata = train_metadata.shape[1]

        for model_name in model_names:

            base_save_path = f'best_base_{model_name}_w_{fold}'
            base_model     = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
            base_model.load_state_dict(torch.load(os.path.join(saved_base_models_folder, base_save_path)))

            for fusion_method in fusion_methods:
                print(f'{"*"*89}\n{model_name.upper()} {metacomb_name.upper()} FOLD {fold} {fusion_method.upper()}\n{"*"*89}\n')

                save_path = f'{model_name}_{fusion_method}_{metacomb_name}_{fold}'
                model     = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)

                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)

                train(model, train_dataloader, val_dataloader, optimizer, scheduler, criterion, device, n_epochs,
                  saved_models_folder, saved_scores_folder, save_path, printfreq=10)

                del model
                gc.collect()
                torch.cuda.empty_cache()

*****************************************************************************************
RESNET50 AGE FOLD 0 CONCAT
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      0.8891       1.0687       0.7260      0.7392            1.0e-03       0.5
    20      0.6410       0.9605       0.7891      0.7493            1.0e-04       0.5
    30      0.6424       0.9200       0.7895      0.7526     ***    1.0e-04       0.5
    40      0.6229       0.9056       0.7919      0.7498     ***    1.0e-05       0.5
    50      0.6170       0.9111       0.7943      0.7438            1.0e-05       0.5
    60      0.6193       0.9086       0.7960      0.7416            1.0e-05       0.5
Training stopped early
--------------------------------------------------------------------------

*****************************************************************************************
VIT_B_32 GENDER FOLD 0 CONCAT
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      0.8715       0.9519       0.7137      0.6865     ***    1.0e-03       0.5
    20      0.8063       0.9525       0.7346      0.6909            1.0e-03       0.5
    30      0.7571       0.9213       0.7445      0.6957     ***    1.0e-04       0.6
    40      0.7386       0.9273       0.7557      0.6988            1.0e-05       0.5
Training stopped early
-----------------------------------------------------------------------------------------
Total time [min] for 45 Epochs: 23.5
*****************************************************************************************
RESNET50 REGION FOLD 0 CON

    60      0.6490       0.9252       0.7835      0.7178            1.0e-06       0.5
    70      0.6537       0.9253       0.7831      0.7233            1.0e-06       0.5
    80      0.6404       0.9199       0.7897      0.7178            1.0e-07       0.5
Training stopped early
-----------------------------------------------------------------------------------------
Total time [min] for 80 Epochs: 41.6
*****************************************************************************************
VIT_B_32 AGE_GENDER FOLD 0 CONCAT
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      1.0879       1.2030       0.6725      0.6036            1.0e-03       0.5
    20      0.7814       0.9315       0.7431      0.7036     ***    1.0e-04       0.5
    30      0.7591       0

Training stopped early
-----------------------------------------------------------------------------------------
Total time [min] for 19 Epochs: 9.9
*****************************************************************************************
RESNEXT50 GENDER_REGION FOLD 0 CONCAT
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      0.6133       0.9175       0.7966      0.7226            1.0e-03       0.5
Training stopped early
-----------------------------------------------------------------------------------------
Total time [min] for 17 Epochs: 8.8
*****************************************************************************************
VGG11 GENDER_REGION FOLD 0 CONCAT
*****************************************************************************************

 Ep

### Modelos faltantes

In [16]:
# model_names = ['vgg11', 'vit_b_32']
model_names = ['vit_b_32']

In [17]:
metacombs = [('age', 'gender', 'region')]

In [18]:
#folds = [0, 1, 2, 3, 4]
#folds = [1, 2, 3, 4]
folds  = [0]
n_reducer_block = 256

for fold in folds:
    for metacomb in metacombs:
        metacomb_name = '_'.join(list(metacomb))
        
        metadata_cols = combcols_dict[metacomb]
        
         # Dataloaders
        train_idcs = train_folds[fold]
        val_idcs   = val_folds[fold]
        train_imgs = df.loc[train_idcs, 'image'].values
        val_imgs   = df.loc[val_idcs, 'image'].values
        test_imgs  = df.loc[test_idcs, 'image'].values

        train_paths = [f'{os.path.join(data_dir, img)}.jpg' for img in train_imgs]
        val_paths   = [f'{os.path.join(data_dir, img)}.jpg' for img in val_imgs]
        test_paths  = [f'{os.path.join(data_dir, img)}.jpg' for img in test_imgs]

        train_labels = df.loc[train_idcs, 'diagnostic_number'].values
        val_labels   = df.loc[val_idcs,   'diagnostic_number'].values
        test_labels  = df.loc[test_idcs,  'diagnostic_number'].values

        train_metadata = df.loc[train_idcs, metadata_cols].values
        val_metadata   = df.loc[val_idcs, metadata_cols].values
        test_metadata  = df.loc[test_idcs, metadata_cols].values
        train_dataloader = get_data_loader(train_paths, train_labels, metadata=train_metadata, transform=train_transform, batch_size=batch_size, num_workers=num_workers)
        val_dataloader   = get_data_loader(val_paths, val_labels, metadata=val_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers)
        test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 

        # Training
        n_classes = len(set(train_labels))
        n_metadata = train_metadata.shape[1]

        for model_name in model_names:

            base_save_path = f'best_base_{model_name}_w_{fold}'
            base_model     = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
            base_model.load_state_dict(torch.load(os.path.join(saved_base_models_folder, base_save_path)))

            for fusion_method in fusion_methods:
                print(f'{"*"*89}\n{model_name.upper()} {metacomb_name.upper()} FOLD {fold} {fusion_method.upper()}\n{"*"*89}\n')

                save_path = f'{model_name}_{fusion_method}_{metacomb_name}_{fold}'
                model     = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)

                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)

                train(model, train_dataloader, val_dataloader, optimizer, scheduler, criterion, device, n_epochs,
                  saved_models_folder, saved_scores_folder, save_path, printfreq=10)

                del model
                gc.collect()
                torch.cuda.empty_cache()

*****************************************************************************************
VIT_B_32 AGE_GENDER_REGION FOLD 0 CONCAT
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      1.1161       1.0959       0.6751      0.6853            1.0e-03       0.6
    20      0.7629       0.9208       0.7496      0.6986     ***    1.0e-04       0.6
    30      0.7515       0.9078       0.7522      0.7082            1.0e-04       0.6
    40      0.7353       0.8867       0.7532      0.7135     ***    1.0e-04       0.6
    50      0.7113       0.8846       0.7631      0.7171            1.0e-05       0.6
    60      0.7140       0.8938       0.7696      0.7125            1.0e-05       0.6
    70      0.7093       0.8960       0.7616      0.7147            1.0e-06       0

## Prueba metablock

In [ ]:
#folds = [0, 1, 2, 3, 4]
#folds = [1, 2, 3, 4]
fusion_methods = ['metablock']
folds  = [0]
n_reducer_block = 256

for fold in folds:
    for metacomb in metacombs:
        metacomb_name = '_'.join(list(metacomb))
        
        metadata_cols = combcols_dict[metacomb]
        
         # Dataloaders
        train_idcs = train_folds[fold]
        val_idcs   = val_folds[fold]
        train_imgs = df.loc[train_idcs, 'image'].values
        val_imgs   = df.loc[val_idcs, 'image'].values
        test_imgs  = df.loc[test_idcs, 'image'].values

        train_paths = [f'{os.path.join(data_dir, img)}.jpg' for img in train_imgs]
        val_paths   = [f'{os.path.join(data_dir, img)}.jpg' for img in val_imgs]
        test_paths  = [f'{os.path.join(data_dir, img)}.jpg' for img in test_imgs]

        train_labels = df.loc[train_idcs, 'diagnostic_number'].values
        val_labels   = df.loc[val_idcs,   'diagnostic_number'].values
        test_labels  = df.loc[test_idcs,  'diagnostic_number'].values

        train_metadata = df.loc[train_idcs, metadata_cols].values
        val_metadata   = df.loc[val_idcs, metadata_cols].values
        test_metadata  = df.loc[test_idcs, metadata_cols].values
        train_dataloader = get_data_loader(train_paths, train_labels, metadata=train_metadata, transform=train_transform, batch_size=batch_size, num_workers=num_workers)
        val_dataloader   = get_data_loader(val_paths, val_labels, metadata=val_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers)
        test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 

        # Training
        n_classes = len(set(train_labels))
        n_metadata = train_metadata.shape[1]
        
        for model_name in model_names:

            base_save_path = f'best_base_{model_name}_w_{fold}'
            base_model     = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)
            base_model.load_state_dict(torch.load(os.path.join(saved_base_models_folder, base_save_path)))

            for fusion_method in fusion_methods:
                print(f'{"*"*89}\n{model_name.upper()} {metacomb_name.upper()} FOLD {fold} {fusion_method.upper()}\n{"*"*89}\n')

                save_path = f'{model_name}_{fusion_method}_{metacomb_name}_{fold}'
                model     = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)

                optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
                scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)

                train(model, train_dataloader, val_dataloader, optimizer, scheduler, criterion, device, n_epochs,
                  saved_models_folder, saved_scores_folder, save_path, printfreq=10)

                del model
                gc.collect()
                torch.cuda.empty_cache()

*****************************************************************************************
RESNET18 AGE FOLD 0 METABLOCK
*****************************************************************************************

 Epoch    Train Loss    Val Loss    Train Acc    Val Acc    Best      lr      Time [min]
-----------------------------------------------------------------------------------------
    10      1.2209       1.1053       0.6307      0.6921     ***    1.0e-03       0.5
    20      1.1470       1.0794       0.6484      0.6923            1.0e-03       0.5
    30      1.0892       1.0071       0.6655      0.7043            1.0e-03       0.5
    40      1.0064       0.9626       0.6911      0.7108            1.0e-04       0.5
    50      0.9863       0.9507       0.7001      0.7099            1.0e-05       0.5
    60      0.9864       0.9578       0.6979      0.6988            1.0e-06       0.5
Training stopped early
-----------------------------------------------------------------------

In [24]:
#os.listdir('imgs/ISIC_2019_Training_Input')

# Testing

In [19]:
fusion_methods

['concat']

In [20]:
folds            = [0]
model_names = ['resnet18', 'resnet50', 'effnetb3', 'resnext50', 'vgg11', 'vit_b_32']
# model_names      = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'effnetb0', 'effnetb1',
#                'effnetb2', 'effnetb3', 'effnetb4', 'effnetb5']
#fusion_methods   = ['concat', 'metanet', 'metablock']
fusion_methods   = ['concat']

In [21]:
usemeta = ['age', 'gender', 'region']
metacombs = get_combs(usemeta)
metacombs

[('age',),
 ('gender',),
 ('region',),
 ('age', 'gender'),
 ('age', 'region'),
 ('gender', 'region'),
 ('age', 'gender', 'region')]

In [22]:
#folds = [0, 1, 2, 3, 4]
#folds = [1, 2, 3, 4]
folds  = [0]
n_reducer_block = 256

n_classes = 8

all_metrics_dict = dict()
for fold in folds:
    fold_dict = dict()
    for metacomb in metacombs:
        metacomb_dict = dict()
        metacomb_name = '_'.join(list(metacomb))
        
        metadata_cols = combcols_dict[metacomb]
        
         # Dataloaders
        test_imgs  = df.loc[test_idcs, 'image'].values

        test_paths  = [f'{os.path.join(data_dir, img)}.jpg' for img in test_imgs]

        test_labels  = df.loc[test_idcs,  'diagnostic_number'].values

        test_metadata  = df.loc[test_idcs, metadata_cols].values
        test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 

        # Training
        n_metadata = test_metadata.shape[1]
        

        for model_name in model_names:
            model_dict = dict()
            base_model     = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)

            for fusion_method in fusion_methods:
                print(f'{"*"*79}\n{model_name.upper()} {metacomb_name.upper()} FOLD {fold} {fusion_method.upper()}\n{"*"*79}\n')
                
                save_path = f'best_{model_name}_{fusion_method}_{metacomb_name}_{fold}'
                
                model = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)
                model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))

                y_true, y_prob, y_pred = get_scores(model, test_dataloader, batch_size, device)
                np.save(f'test_scores/y_true_{model_name}_{fusion_method}_{metacomb_name}_{fold}', y_true)
                np.save(f'test_scores/y_prob_{model_name}_{fusion_method}_{metacomb_name}_{fold}', y_prob)
                np.save(f'test_scores/y_pred_{model_name}_{fusion_method}_{metacomb_name}_{fold}', y_pred)
                
                metrics_dict = get_metrics(y_true, y_prob, y_pred)
                
                del model
                gc.collect()
                torch.cuda.empty_cache()
                
                model_dict[fusion_method] = metrics_dict
            metacomb_dict[model_name] = model_dict
        fold_dict[metacomb_name] = metacomb_dict
    all_metrics_dict[fold] = fold_dict

*******************************************************************************
RESNET18 AGE FOLD 0 CONCAT
*******************************************************************************



/home/gabriel/skin/ISIC2019/../utils/metrics.py:47: UserWarning: An output with one or more elements was resized since it had shape [4968, 6], which does not match the required output shape [4968, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(prob_list, out=y_prob)


*******************************************************************************
RESNET50 AGE FOLD 0 CONCAT
*******************************************************************************



/home/gabriel/skin/ISIC2019/../utils/metrics.py:47: UserWarning: An output with one or more elements was resized since it had shape [4968, 6], which does not match the required output shape [4968, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(prob_list, out=y_prob)


Loaded pretrained weights for efficientnet-b3
*******************************************************************************
EFFNETB3 AGE FOLD 0 CONCAT
*******************************************************************************



...

/home/gabriel/skin/ISIC2019/../utils/metrics.py:47: UserWarning: An output with one or more elements was resized since it had shape [4968, 6], which does not match the required output shape [4968, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(prob_list, out=y_prob)


*******************************************************************************
VGG11 AGE_GENDER_REGION FOLD 0 CONCAT
*******************************************************************************



/home/gabriel/skin/ISIC2019/../utils/metrics.py:47: UserWarning: An output with one or more elements was resized since it had shape [4968, 6], which does not match the required output shape [4968, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(prob_list, out=y_prob)


*******************************************************************************
VIT_B_32 AGE_GENDER_REGION FOLD 0 CONCAT
*******************************************************************************



/home/gabriel/skin/ISIC2019/../utils/metrics.py:47: UserWarning: An output with one or more elements was resized since it had shape [4968, 6], which does not match the required output shape [4968, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:17.)
  torch.cat(prob_list, out=y_prob)


In [23]:
import shutil
shutil.make_archive('test_scores', 'zip', 'test_scores')

'/home/gabriel/skin/ISIC2019/test_scores.zip'

In [22]:
# #folds = [0, 1, 2, 3, 4]
# #folds = [1, 2, 3, 4]
# folds  = [0]
# n_reducer_block = 256

# all_metrics_dict = dict()
# for fold in folds:
#     fold_dict = dict()
#     for metacomb in metacombs:
#         metacomb_name = '_'.join(list(metacomb))
        
#         metadata_cols = combcols_dict[metacomb]
        
#          # Dataloaders
#         test_imgs  = df.loc[test_idcs, 'image'].values

#         test_paths  = [f'{os.path.join(data_dir, img)}.jpg' for img in test_imgs]

#         test_labels  = df.loc[test_idcs,  'diagnostic_number'].values

#         test_metadata  = df.loc[test_idcs, metadata_cols].values
#         test_dataloader  = get_data_loader(test_paths, test_labels, metadata=test_metadata, transform=val_transform, batch_size=batch_size, num_workers=num_workers) 

#         # Training
#         n_classes = len(set(train_labels))
#         n_metadata = test_metadata.shape[1]

#         for model_name in model_names:
#             model_dict = dict()
#             base_model     = BaseMetaModel(get_model(model_name, n_classes=n_classes, pretrained=True)).to(device)

#             for fusion_method in fusion_methods:
#                 print(f'{"*"*79}\n{model_name.upper()} {metacomb_name.upper()} FOLD {fold} {fusion_method.upper()}\n{"*"*79}\n')
                
#                 save_path = f'best_{model_name}_{fusion_method}_{metacomb_name}_{fold}'
#                 #print(save_path, n_metadata)
                
#                 model = MetaModel(base_model, n_classes, n_metadata=n_metadata, fusion_method=fusion_method, n_reducer_block=n_reducer_block).to(device)
#                 model.load_state_dict(torch.load(os.path.join(saved_models_folder, save_path)))

#                 y_true, y_prob, y_pred = get_scores(model, test_dataloader, batch_size, device)
#                 metrics_dict = get_metrics(y_true, y_prob, y_pred)
                
#                 del model
#                 gc.collect()
#                 torch.cuda.empty_cache()
                
#                 model_dict[fusion_method] = metrics_dict
#             metacomb_dict[model_name] = model_dict
#         fold_dict[metacomb_name] = metacomb_dict
#     all_metrics_dict[fold] = fold_dict

In [ ]:
import json

with open('metrics_combs_allmodels_concat.json', 'w') as outfile:
    json.dump(all_metrics_dict, outfile)
    
all_metrics_dict

In [26]:
pd.DataFrame(all_metrics_dict[0]['age']['resnet18'])

,concat
accuracy,0.730676
auc,0.929802
balanced_accuracy,0.520147
f1-score,0.725528
precision,0.724709
recall,0.730676
support,4968.000000


In [25]:
metacomb_names = ['_'.join(list(metacomb)) for metacomb in metacombs]
metacomb_names

['age',
 'gender',
 'region',
 'age_gender',
 'age_region',
 'gender_region',
 'age_gender_region']

In [26]:
pd.DataFrame({metacomb: all_metrics_dict[0][metacomb]['resnet18']['concat'] for metacomb in metacomb_names})

,age,gender,region,age_gender,age_region,gender_region,age_gender_region
precision,0.716968,0.695939,0.711760,0.720001,0.724570,0.690627,0.724266
recall,0.724638,0.707729,0.716586,0.726651,0.730878,0.705113,0.730676
f1-score,0.719229,0.700178,0.712359,0.721029,0.725764,0.694337,0.725577
support,4968.000000,4968.000000,4968.000000,4968.000000,4968.000000,4968.000000,4968.000000
accuracy,0.724638,0.707729,0.716586,0.726651,0.730878,0.705113,0.730676
balanced_accuracy,0.511740,0.478285,0.520425,0.506165,0.517731,0.476470,0.527001
auc,0.926833,0.917472,0.920209,0.927797,0.929533,0.914203,0.929182


In [29]:
# pd.DataFrame({metacomb: all_metrics_dict[0][metacomb]['resnet18']['metanet'] for metacomb in metacomb_names})

In [30]:
# pd.DataFrame({metacomb: all_metrics_dict[0][metacomb]['resnet18']['metablock'] for metacomb in metacomb_names})